In [1]:
using Unitful

# WiFi Headphones project

The idea is to have wireless headphones using the ESP32 to recieve audio and then transmit it in a small package

## Design Parameters

- Battery operated, 2-3h
- USB-C Charging
- External antenna port available
- Decent audio in-out using combo jack

## Components

- MCU: `ESP32-S2FH4`
- Battery Charging: `BQ21062`
- LDO: `TLV758P`
- Audio Codec: `SGTL5000`
- Battery: 3.7V li-po 34 * 50mm (using `L573450`)

## Power estimation

MCU: 190mA @ 3.3V as per Table 15 in datasheet (pg 37)
Codec: 3-5mA @ vbat

assume 220mA worst case battery draw (at battery voltage because LDO)

3h requires 660mAh battery

Choose battery: 34 * 50mm size (common size)

980mAh Cellevia L573450 li-po available from TME (5.7mm thick)

Stack PCB on top of battery for compact design, so 34 * 50mm pcb

## LDO

Need to maintain 3.6V maximum from ESP32 and other downstream chips, but lithium batteries can have up to 4.2V, and PMID will regulate to 4.5V when charging. Connecting directly to components would damage chips, but the built-in LDO in the TLV758P can only handle 150mA, too low for the ESP32-S2 when using WiFi.

Regulating to 3.6V is preferred over 3.3V, since this will allow the headphone amp in the SGTL5000 to drive higher-impedance headphones, and with more power. The LDO is expected to operate in dropout for significant periods, acting as a "maximum voltage limit" more than a traditional voltage regulator. For this reason, low dropout voltage is beneficial. It also means a requirement for an adjustible voltage regulator, since there are no 3.6V preset LDOs.

The TLV758P was chosen for it's low dropout voltage (<100mV typ at 3.3V+), low cost, small size, and low package thermal resistance (it has a power ground pad).

The output voltage is set at the FB pin, using the equation

$V_{OUT} = 0.55\textrm{V} \times (1 + \frac{R_1}{R_2})$

The LDO has an output voltage accuracy of 1%, so we'll aim for 3.55V to avoid breaching 3.6V. 0.1% accuracy resistors will be used to avoid breaking 3.6V.

The equation can be rearranged in terms of $\frac{R_1}{R_2}$ like so:

$\frac{R_1}{R_2} = \frac{V_{OUT}}{0.55\textrm{V}} - 1$

In [2]:
Vout = 3.55u"V"

(Vout/0.55u"V") - 1

5.454545454545454

Putting 5.454545 into the calculator [here](http://jansson.us/resistors.html) gives us the best ratio of resistors in the E series: 120:22

0603 0.1% resistors are only available up to 1Mohm, so that gives us a 120k/22k resistor divider.

$R_1 = 120\textrm{k}\Omega$

$R_2 = 22\textrm{k}\Omega$

To calculate the worst case upper bound of voltage, we use the upper bound of R1, and the lower bound of R1, multiplied by the worst case 1% accuracy of the LDO:

$1.01 \times 0.55\textrm{V} \times (1 + \frac{1.001R_1}{0.999R_2})$

In [3]:
R1 = 120u"kΩ"
R2 = 22u"kΩ"

1.01 * 0.55u"V" * (1 + (1.001*R1)/(0.999*R2))

3.591566066066066 V

The voltage stays under 3.6V.

The datasheet specifies that the feedback pin has a current of 10nA, and that the divider current must be 100 times the feedback current to ignore the impedance of the feedback pin. Therefore the current through the resistor divider must be greater than 1µA. The current through the resistor divider is $\frac{3.6\textrm{V}}{R_1+R_2}$

In [4]:
u"µA"(3.6u"V" / (R1+R2))

25.35211267605634 μA

Therefore, the values of R1 and R2 are justified.

A feed-forward capacitor of 100nF was selected by looking at figure 31 and 34 of the datasheet.

## TODO

- Figure out power button and off/standby mode